In [1]:
import rospy
import math
import numpy as np
import cv2
import time

from sensor_msgs.msg import Image
from aruco_msgs.msg import MarkerArray
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from operator import itemgetter

bridge = CvBridge()

In [2]:
rospy.init_node('turtlebot_singleplayer', anonymous=True)
rospy.sleep(0.5)
lower_blue = (0,0,120)
upper_blue = (100,100,255)
lower_red = (200,0,0)
upper_red = (255,100,100)

In [3]:
def callback_color(msg):
    
    #start_time = time.time()
    
    global x_ball
    global y_ball
    global dist_ball
    color_timestamp = msg.header.stamp
    color_img = bridge.imgmsg_to_cv2(msg, "rgb8")
    mask_blue = cv2.inRange(color_img, lower_blue, upper_blue)
    mask_red = cv2.inRange(color_img, lower_red, upper_red)
    kernel = np.ones((5,5),np.uint8)
    erosion_blue = cv2.erode(mask_blue,kernel,iterations = 1)
    dilation_blue = cv2.dilate(erosion_blue,kernel,iterations = 3)
    erosion_red = cv2.erode(mask_red,kernel,iterations = 1)
    dilation_red = cv2.dilate(erosion_red,kernel,iterations = 3)
    mask = cv2.bitwise_or(dilation_blue, dilation_red)
    im, cnt, hrc = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt.sort(key=cv2.contourArea, reverse=True)
    if cnt:
        x, y, w, h = cv2.boundingRect( cnt[0] )
        if (w*h > 2000):
            x_ball = x+w/2
            y_ball = y+h/2
            dist_ball = np.nanmean(depth_img[y:y+h,x:x+w])
        else:
            x_ball = 0
            y_ball = 0
            dist_ball = None
    else:
        x_ball = 0
        y_ball = 0
        dist_ball = None
    
    #print "time %f"%(time.time() - start_time)
    
    #cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255), 2)
    #plt.imshow(img);

In [4]:
def callback_depth(msg):
    global depth_img
    global depth_timestamp
    depth_timestamp = msg.header.stamp
    depth_img = bridge.imgmsg_to_cv2(msg, "32FC1")

In [3]:
def callback_odom(data):
    global x
    global y
    global theta
    position = data.pose.pose.position
    orientation = data.pose.pose.orientation
    x = round(position.x, 3)
    y = round(position.y, 3)
    theta = round(2*math.atan2(orientation.z,orientation.w), 3)
    if theta < 0:
        theta = 2*math.pi + theta

In [6]:
def callback_image(msg):
    global marker_img
    global marker_timestamp
    marker_img_timestamp = msg.header.stamp
    marker_img = bridge.imgmsg_to_cv2(msg, "rgb8")

def callback_array(msg):
    global marker_array
    global marker_array_timestamp
    marker_array_timestamp = msg.header.stamp
    marker_array = msg.markers


In [4]:
#turn to certain theta
def turnToTheta(theta_tar):
    twist = Twist()
    twist.linear.x = 0.0
    ang_diff = theta_tar - theta
    s = np.sign(ang_diff)
    if abs(ang_diff) > math.pi:
        s *= -1.0
        ang_diff = 2*math.pi - abs(ang_diff)
    ang_vel = s*ang_diff*0.6
    if abs(ang_vel) < 0.2:
        ang_vel = np.sign(ang_vel)*0.2
    twist.angular.z = ang_vel
    r = rospy.Rate(10)
    while(abs(theta_tar-theta) > 0.1):
        publisher.publish(twist)
        r.sleep()
        ang_diff = abs(theta_tar - theta)
        if ang_diff > math.pi:
            ang_diff = 2*math.pi - ang_diff
        ang_vel = s*ang_diff*0.6
        if abs(ang_vel) < 0.2:
            ang_vel = np.sign(ang_vel)*0.2
        twist.angular.z = ang_vel
        #print theta

In [8]:
subscriber_color = rospy.Subscriber("/camera/rgb/image_rect_color", Image, callback_color)
subscriber_depth = rospy.Subscriber("/camera/depth/image_rect", Image, callback_depth)
subscriber = rospy.Subscriber("odom", Odometry, callback_odom)
publisher = rospy.Publisher('cmd_vel_mux/input/teleop', Twist, queue_size=10)
subscriber_array = rospy.Subscriber("/aruco_marker_publisher/markers", MarkerArray, callback_array)

[ERROR] [1510944161.443425]: bad callback: <function callback_color at 0x7f782805cb90>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-3-6e3d4d883edb>", line 25, in callback_color
    dist_ball = np.nanmean(depth_img[y:y+h,x:x+w])
NameError: global name 'depth_img' is not defined

[ERROR] [1510944161.467878]: bad callback: <function callback_color at 0x7f782805cb90>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-3-6e3d4d883edb>", line 25, in callback_color
    dist_ball = np.nanmean(depth_img[y:y+h,x:x+w])
NameError: global name 'depth_img' is not defined

[ERROR] [1510944161.490900]: bad callback: <function callback_color at 0x7f782805cb90>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py

In [5]:
subscriber = rospy.Subscriber("odom", Odometry, callback_odom)
publisher = rospy.Publisher('cmd_vel_mux/input/teleop', Twist, queue_size=10)

In [9]:
#Turn around look for the ball and the gate
gate_error, gate_dist, gate_theta = [],[],[] # Whether it was found, and its theta coordinate, and x 
twist = Twist()
#We need this if we want to turn full circle
theta_initial = theta
half_circle = False
marker_array = None
m = None
#Find gate
while (abs(theta-theta_initial)>0.1 or (half_circle != True)):
    if(theta - 6.1 > 0):
        half_circle = True
    twist.angular.z = 0.2
    r = rospy.Rate(10)
    for i in range(5):
        publisher.publish(twist)
        r.sleep()
        
    try:
        m = marker_array[0]
        
        
        gate_error.append(abs(m.pose.pose.orientation.x))
        gate_dist.append(abs(m.pose.pose.orientation.z))
        gate_theta.append(theta)
        print m.pose.pose.orientation.x,m.pose.pose.orientation.y,m.pose.pose.orientation.z
        marker_array = None
    except TypeError:
        continue

index = min(enumerate(gate_error), key=itemgetter(1))[0]
theta_gate = gate_theta[index]
dist_gate = gate_dist[index]

0.170935919023 0.743387354005 -0.620773588724
0.120398492566 0.75774814588 -0.627196350316
-0.189839360207 -0.666273905329 0.702779560884
0.121567406125 0.756511782828 -0.63055155171
0.0622066205621 0.759916834366 -0.64368580786
0.0835253242947 0.780978203943 -0.613203076619
-0.138358403236 -0.662402616643 0.728339861546
0.02956548262 0.747710031247 -0.663266536298


/usr/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


0.00327019988542 0.73968878737 -0.672794382028
-0.0912292343162 -0.69121231522 0.713788161389
0.0494286424618 0.71773036058 -0.694188763935
-0.065535259192 0.723436305532 -0.681432339422
-0.10411354253 0.742093286521 -0.649612555775
-0.105271647462 0.708138761159 -0.68639743306
-0.133989957062 0.722026899165 -0.65855757592
0.146753594491 -0.685842347021 0.692175011922
-0.183548491969 0.689919414794 -0.666535912482
-0.207688229009 0.697443816976 -0.639127535988


In [23]:
turnToTheta(theta_gate)

In [10]:
#Find ball
ball = [False, 0,0]
twist = Twist()
#We need this if we want to turn full circle
theta_initial = theta
half_circle = False

ball_pos = []
theta_list = []
while (abs(theta-theta_initial)>0.1 or (half_circle != True)):
    if(theta - 6.1 > 0):
        half_circle = True
        
    twist.angular.z = 0.2
    r = rospy.Rate(10)
    for i in range(5):
        publisher.publish(twist)
        r.sleep()
    if x_ball:
        if not dist_ball:
            dist_ball = 100
        ball_pos.append([x_ball, y_ball, theta, dist_ball])
        theta_list.append(theta)
        print dist_ball, theta
ball_np = np.array(ball_pos)
idx = (np.abs(ball_np[:,0]-320)).argmin()
#idx = (np.abs(ball_np[:,3])).argmin()
ball_pos = ball_np[idx]
ball_dist = np.nanmin(ball_np[:,3])
ball_theta = ball_pos[2]
print ball_pos

#Theta cleanup
sd = np.std(ball_np[:,2])
mean = np.mean(ball_np[:,2])
final_list = [x for x in theta_list if (x > mean - sd)]
final_list = [x for x in final_list if (x < mean +  sd)]
ball_theta = sum(final_list) / float(len(final_list))

3.13657 3.168
1.66072 3.644
0.876174 3.752
0.69525 3.85
0.735691 3.943
0.738258 4.034
0.749024 4.134
0.763446 4.232
0.759297 4.323
0.74719 4.423
0.748286 4.524
0.705621 4.629
0.693905 4.722
0.650337 4.81
0.708471 4.918
1.55517 5.016
1.84184 0.469
2.11166 0.971
2.12044 1.052
1.98791 1.153
1.94789 1.257
1.85715 1.351
1.91827 1.449
1.04035 1.544
1.0344 1.648
nan 2.348
2.96835 2.434
3.20977 2.52
3.31317 2.622
3.37855 2.736
3.4579 2.851
3.43523 2.935
3.35496 3.039
[ 322.          172.            2.935         3.43523264]


In [14]:
turnToTheta(ball_theta)

[ERROR] [1510942697.531366]: bad callback: <function callback_color at 0x7faa64d2b9b0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-4-f23636942a4c>", line 3, in callback_color
    start_time = time.time()
AttributeError: 'int' object has no attribute 'time'

[ERROR] [1510942697.566668]: bad callback: <function callback_color at 0x7faa64d2b9b0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-4-f23636942a4c>", line 3, in callback_color
    start_time = time.time()
AttributeError: 'int' object has no attribute 'time'

[ERROR] [1510942697.634348]: bad callback: <function callback_color at 0x7faa64d2b9b0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(

In [11]:
#calculate angle and distance to go to he position to kick the ball
dist_BT = 0.5
ang_GRB = abs(theta_gate - ball_theta)
dist_RB = ball_dist
dist_RG = dist_gate
dist_BG = math.sqrt(pow(dist_RG,2) + pow(dist_RB,2) - 2*dist_RG*dist_RB*math.cos(ang_GRB))
ang_GBR = math.acos((pow(dist_BG,2) + pow(dist_RB,2) - pow(dist_RG,2)) / (2*dist_BG*dist_RB))
ang_RBT = math.pi - ang_GBR
dist_RT = math.sqrt(pow(dist_RB,2) + pow(dist_BT,2) - 2*dist_RB*dist_BT*math.cos(ang_RBT))
ang_BRT = math.acos((pow(dist_RB,2) + pow(dist_RT,2) - pow(dist_BT,2)) / (2*dist_RB*dist_RT))
ang_BTR = math.acos((pow(dist_BT,2) + pow(dist_RT,2) - pow(dist_RB,2)) / (2*dist_BT*dist_RT))
ang_back = math.pi - ang_BTR
if ball_theta < theta_gate:
    theta_tar = ball_theta - ang_BRT
    theta_back = theta_tar + ang_back
else:
    theta_tar = ball_theta + ang_BRT
    theta_back = theta_tar - ang_back


In [12]:
#move to the target position
turnToTheta(theta_tar)
time = int(round(dist_RT*40))
twist = Twist()
twist.linear.x = 0.2
twist.angular.z = 0.0
r = rospy.Rate(10)
for i in range(time):
    publisher.publish(twist)
    r.sleep()

In [24]:
turnToTheta(theta_back)

In [9]:
#find the ball
print [x_ball, y_ball]
twist = Twist()
twist.linear.x = 0.0
twist.angular.z = -0.3
r = rospy.Rate(10)
while x_ball == 0:
    for i in range(5):
        publisher.publish(twist)
        r.sleep()
print "found the ball!!"
print [x_ball, y_ball]

[0, 0]
found the ball!!


In [15]:
def getAngleDiff():
    x_theta = x_ball - img_w/2
    y_theta = img_h - y_ball
    angle = math.pi/2 - math.atan2(y_theta,x_theta)
    return angle

In [11]:
getAngleDiff()

-0.3337981642508283

In [16]:
#turn to ball
print (x_odom,y_odom,theta_odom)
twist = Twist()
angle = getAngleDiff()
twist.linear.x = 0.0
if angle > 0:
    twist.angular.z = -0.2
else:
    twist.angular.z = 0.2
r = rospy.Rate(10)
while abs(angle) > 0.1:
    for i in range(5):
        publisher.publish(twist)
        r.sleep()
    angle = getAngleDiff()
print "finish", (x_odom,y_odom,theta_odom)

(0.447, 0.296, 0.679)
1.05416804189
1.05416804189
1.05711463903
1.05711463903
1.05711463903
1.05416804189
1.05416804189
0.906464897965
0.906464897965
0.141897054604
0.141897054604
0.141897054604
-0.70690806076
finish (0.448, 0.296, -0.763)
not big enough 407.0
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball
no ball


In [14]:
print theta_gate
print ball_theta

3.596
3.32735294118


In [26]:
print theta_back

2.62554277113
